In [2]:
pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.3 MB/s eta 0:00:00


In [3]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW


In [4]:
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

In [5]:
def __len__(self):
        return len(self.data)


In [6]:
def __getitem__(self, index):
        text = str(self.data.loc[index, "text"])
        label = self.data.loc[index, "label"]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].flatten()
        attention_mask = encoding["attention_mask"].flatten()
        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "label": torch.tensor(label, dtype=torch.long)
        }

In [7]:
from google.colab import files
uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [8]:
import pandas as pd
import io

df_ = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))
df = df_.copy()

df.columns = [col.upper() for col in df.columns]
df.head()

,TEXT,LABEL
0,I love using BERT for natural language process...,1
1,BERT is a powerful pre-trained model.,1
2,This sentence is unrelated to BERT.,0
3,BERT can handle various NLP tasks.,1


In [9]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [12]:
# Set the maximum sequence length
max_length = 128

# Create the custom dataset
dataset = CustomDataset(df, tokenizer, max_length)

# Create the data loader
batch_size = 4
dataloader = DataLoader(df, batch_size=batch_size, shuffle=True)
print(type(dataloader))
# Initialize the BERT model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# Set the device (GPU or CPU)
device = torch.device("cpu")
model.to(device)

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)
no_deprecation_warning=True

# Training loop
epochs = 5
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for i in dataloader:
        input_ids = i["input_ids"].to(device)
        attention_mask = i["attention_mask"].to(device)
        labels = i["label"].to(device)

        # Zero the gradients
        model.zero_grad()

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters
        optimizer.step()

        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)
    print(f"Epoch {epoch + 1}/{epochs} - Loss: {average_loss:.4f}")

# Save the trained model
model.save_pretrained("saved_model")
tokenizer.save_pretrained("saved_model")



<class 'torch.utils.data.dataloader.DataLoader'>


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

KeyError: ignored